In [29]:
!python -m pip install langchain==0.1 langchain_openai python-dotenv chromadb docx2txt -q

You should consider upgrading via the 'c:\Users\SERKAN\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [30]:
import os
from dotenv import load_dotenv, find_dotenv,dotenv_values
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate


In [31]:
config = dict(dotenv_values("azureConfig.env"))

In [32]:
os.environ["OPENAI_API_TYPE"] = config["OPENAI_API_TYPE"]
os.environ["AZURE_OPENAI_ENDPOINT"] = config["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = config["OPENAI_API_VERSION"]

In [33]:
model_deployment_name = os.getenv("gpt-35-turbo")
embedding_deployment_name = os.getenv("text-embedding-ada-002")

In [34]:
loader = Docx2txtLoader(r"C:\Users\SERKAN\Desktop\DİLARA/Yeni Microsoft Word Belgesi2.docx")
data = loader.load()

In [35]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=0
)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

67


In [36]:
embedding = AzureOpenAIEmbeddings(
    deployment=embedding_deployment_name,
    )

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)

In [37]:
question = "Emekliye ayırma planlaması nasıl olur"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [41]:
llm = AzureChatOpenAI(
    deployment_name=model_deployment_name,
    model_name = "test",
    temperature=0
)

In [42]:
template = """Use the following pieces of context to answer the question at the end.
Use three sentences maximum and keep the answer as concise as possible.
Don't try to make up the answer, only use the context to answer the question.
The pieces of context refer to the this word file.

Context:
{context}

Question: {question}
Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

retriever = vectorstore.as_retriever(search_kwargs={"k": 12})

qa_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


response = qa_chain.invoke(question)
response

'Emekliye ayırma planlaması, iş sürekliliği planları, geri çekilme planı, yapılandırma ve yeniden kurulum prosedürleri gibi ayrıntılı adımlar içermelidir. Emeklilik öncesi iş süreci belgelenmeli ve sürecin perspektiflerinden anlaşılmalıdır. Emeklilik infazının zamanlaması dikkatle düşünülmeli ve program dahil edilmelidir.'